<a href="https://colab.research.google.com/github/MBrandao07/Case_Porto_Seguro_Kaggle/blob/main/Case_Porto_Seguro_Escoragem.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import itertools
import pickle
from sklearn.metrics import confusion_matrix, roc_curve, precision_recall_curve, roc_auc_score

import warnings
warnings.filterwarnings('ignore')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
df_train_00 = pd.read_csv('/content/drive/MyDrive/1 - Aulas PoD Academy/Cientista de Dados POD/Aulas/Case Porto Seguro/train.csv')
df_train_00.shape

(595212, 59)

In [ ]:
df_train_00.head()

,id,target,ps_ind_01,ps_ind_02_cat,ps_ind_03,ps_ind_04_cat,ps_ind_05_cat,ps_ind_06_bin,ps_ind_07_bin,ps_ind_08_bin,...,ps_calc_11,ps_calc_12,ps_calc_13,ps_calc_14,ps_calc_15_bin,ps_calc_16_bin,ps_calc_17_bin,ps_calc_18_bin,ps_calc_19_bin,ps_calc_20_bin
0,7,0,2,2,5,1,0,0,1,0,...,9,1,5,8,0,1,1,0,0,1
1,9,0,1,1,7,0,0,0,0,1,...,3,1,1,9,0,1,1,0,1,0
2,13,0,5,4,9,1,0,0,0,1,...,4,2,7,7,0,1,1,0,1,0
3,16,0,0,1,2,0,0,1,0,0,...,2,2,4,9,0,0,0,0,0,0
4,17,0,0,2,0,1,0,1,0,0,...,3,1,1,3,0,0,0,1,1,0


In [ ]:
df_test_00 = pd.read_csv('/content/drive/MyDrive/1 - Aulas PoD Academy/Cientista de Dados POD/Aulas/Case Porto Seguro/test.csv')
df_test_00.shape

(892816, 58)

In [ ]:
df_test_00.head()

,id,ps_ind_01,ps_ind_02_cat,ps_ind_03,ps_ind_04_cat,ps_ind_05_cat,ps_ind_06_bin,ps_ind_07_bin,ps_ind_08_bin,ps_ind_09_bin,...,ps_calc_11,ps_calc_12,ps_calc_13,ps_calc_14,ps_calc_15_bin,ps_calc_16_bin,ps_calc_17_bin,ps_calc_18_bin,ps_calc_19_bin,ps_calc_20_bin
0,0,0,1,8,1,0,0,1,0,0,...,1,1,1,12,0,1,1,0,0,1
1,1,4,2,5,1,0,0,0,0,1,...,2,0,3,10,0,0,1,1,0,1
2,2,5,1,3,0,0,0,0,0,1,...,4,0,2,4,0,0,0,0,0,0
3,3,0,1,6,0,0,1,0,0,0,...,5,1,0,5,1,0,1,0,0,0
4,4,5,1,7,0,0,0,0,0,1,...,4,0,0,4,0,1,1,0,0,1


In [ ]:
# criando novos dataframes baseado no original
df_train_01 = df_train_00.copy()
df_test_01 = df_test_00.copy()

# substituindo -1 por 'missing' em todo o dataframe
df_train_01.replace(-1, np.nan, inplace=True)
df_test_01.replace(-1, np.nan, inplace=True)

In [ ]:
# carregando lista de variáveis que foram excluidas por execesso de nulos
with open('/content/drive/MyDrive/1 - Aulas PoD Academy/Cientista de Dados POD/Aulas/Case Porto Seguro/prd_drop_nullvars.pkl', 'rb') as f:
    loaded_drop_null_feat = pickle.load(f)
loaded_drop_null_feat

['ps_car_03_cat']

In [ ]:
# retirando as variáveis excluídas por nulos dos dfs de teste e treino
df_train_02 = df_train_01.drop(axis=1,columns=loaded_drop_null_feat)
df_test_02 = df_test_01.drop(axis=1,columns=loaded_drop_null_feat)

In [ ]:
df_test_02.head()

,id,ps_ind_01,ps_ind_02_cat,ps_ind_03,ps_ind_04_cat,ps_ind_05_cat,ps_ind_06_bin,ps_ind_07_bin,ps_ind_08_bin,ps_ind_09_bin,...,ps_calc_11,ps_calc_12,ps_calc_13,ps_calc_14,ps_calc_15_bin,ps_calc_16_bin,ps_calc_17_bin,ps_calc_18_bin,ps_calc_19_bin,ps_calc_20_bin
0,0,0,1.0,8,1.0,0.0,0,1,0,0,...,1,1,1,12,0,1,1,0,0,1
1,1,4,2.0,5,1.0,0.0,0,0,0,1,...,2,0,3,10,0,0,1,1,0,1
2,2,5,1.0,3,0.0,0.0,0,0,0,1,...,4,0,2,4,0,0,0,0,0,0
3,3,0,1.0,6,0.0,0.0,1,0,0,0,...,5,1,0,5,1,0,1,0,0,0
4,4,5,1.0,7,0.0,0.0,0,0,0,1,...,4,0,0,4,0,1,1,0,0,1


In [ ]:
# Retirar ID e Target das tabelas
df_train_02 = df_train_02.drop(axis=1, columns=['id','target'])
df_test_02 = df_test_02.drop(axis=1, columns=['id'])

df_test_02.shape,df_train_02.shape

((892816, 56), (595212, 56))

In [ ]:
df_train_02.head()

,ps_ind_01,ps_ind_02_cat,ps_ind_03,ps_ind_04_cat,ps_ind_05_cat,ps_ind_06_bin,ps_ind_07_bin,ps_ind_08_bin,ps_ind_09_bin,ps_ind_10_bin,...,ps_calc_11,ps_calc_12,ps_calc_13,ps_calc_14,ps_calc_15_bin,ps_calc_16_bin,ps_calc_17_bin,ps_calc_18_bin,ps_calc_19_bin,ps_calc_20_bin
0,2,2.0,5,1.0,0.0,0,1,0,0,0,...,9,1,5,8,0,1,1,0,0,1
1,1,1.0,7,0.0,0.0,0,0,1,0,0,...,3,1,1,9,0,1,1,0,1,0
2,5,4.0,9,1.0,0.0,0,0,1,0,0,...,4,2,7,7,0,1,1,0,1,0
3,0,1.0,2,0.0,0.0,1,0,0,0,0,...,2,2,4,9,0,0,0,0,0,0
4,0,2.0,0,1.0,0.0,1,0,0,0,0,...,3,1,1,3,0,0,0,1,1,0


In [ ]:
# carregando as medias das variaveis da base de treino
with open('/content/drive/MyDrive/1 - Aulas PoD Academy/Cientista de Dados POD/Aulas/Case Porto Seguro/prd_fillna.pkl', 'rb') as f:
  loaded_means = pickle.load(f)

def pod_custom_fillna_prod(df, means):
  df.replace(-1, np.nan, inplace=True)
  for col, mean_value in means.items():
      df[col].fillna(mean_value, inplace=True)

  categorical_cols = df.select_dtypes(include=['object']).columns
  df[categorical_cols] = df[categorical_cols].fillna('POD_VERIFICAR')

  return df

# substituindo os nulos pelas médias na base de treino
df_train_02 = pod_custom_fillna_prod(df_train_02,loaded_means)

# substituindo os nulos pelas médias na base de teste
df_test_02 = pod_custom_fillna_prod(df_test_02,loaded_means)


df_test_02.head()

,ps_ind_01,ps_ind_02_cat,ps_ind_03,ps_ind_04_cat,ps_ind_05_cat,ps_ind_06_bin,ps_ind_07_bin,ps_ind_08_bin,ps_ind_09_bin,ps_ind_10_bin,...,ps_calc_11,ps_calc_12,ps_calc_13,ps_calc_14,ps_calc_15_bin,ps_calc_16_bin,ps_calc_17_bin,ps_calc_18_bin,ps_calc_19_bin,ps_calc_20_bin
0,0,1.0,8,1.0,0.0,0,1,0,0,0,...,1,1,1,12,0,1,1,0,0,1
1,4,2.0,5,1.0,0.0,0,0,0,1,0,...,2,0,3,10,0,0,1,1,0,1
2,5,1.0,3,0.0,0.0,0,0,0,1,0,...,4,0,2,4,0,0,0,0,0,0
3,0,1.0,6,0.0,0.0,1,0,0,0,0,...,5,1,0,5,1,0,1,0,0,0
4,5,1.0,7,0.0,0.0,0,0,0,1,0,...,4,0,0,4,0,1,1,0,0,1


In [ ]:
df_train_03 = df_train_02
df_test_03 = df_test_02

In [ ]:
# carregando o scaler
with open('/content/drive/MyDrive/1 - Aulas PoD Academy/Cientista de Dados POD/Aulas/Case Porto Seguro/prd_scaler.pkl', 'rb') as f:
    loaded_scaler = pickle.load(f)

# aplicando na base de treino
df_train_04_scaled = loaded_scaler.transform(df_train_03)
df_train_05 = pd.DataFrame(df_train_04_scaled, columns=df_train_03.columns, index=df_train_03.index)

# aplicando na base de teste
df_test_04s = loaded_scaler.transform(df_test_03)
df_test_05 = pd.DataFrame(df_test_04s, columns=df_test_03.columns, index=df_test_03.index)
df_test_05.head()

,ps_ind_01,ps_ind_02_cat,ps_ind_03,ps_ind_04_cat,ps_ind_05_cat,ps_ind_06_bin,ps_ind_07_bin,ps_ind_08_bin,ps_ind_09_bin,ps_ind_10_bin,...,ps_calc_11,ps_calc_12,ps_calc_13,ps_calc_14,ps_calc_15_bin,ps_calc_16_bin,ps_calc_17_bin,ps_calc_18_bin,ps_calc_19_bin,ps_calc_20_bin
0,-0.958565,-0.542601,1.325015,1.182765,-0.31183,-0.806035,1.701191,-0.442517,-0.477413,-0.019166,...,-1.902556,-0.367316,-1.105234,1.624615,-0.373426,0.770766,0.897048,-0.634586,-0.730772,2.350925
1,1.057180,0.966428,0.213366,1.182765,-0.31183,-0.806035,-0.587823,-0.442517,2.094620,-0.019166,...,-1.474199,-1.198881,0.075186,0.896386,-0.373426,-1.297410,0.897048,1.575831,-0.730772,2.350925
2,1.561116,-0.542601,-0.527733,-0.845588,-0.31183,-0.806035,-0.587823,-0.442517,2.094620,-0.019166,...,-0.617485,-1.198881,-0.515024,-1.288303,-0.373426,-1.297410,-1.114768,-0.634586,-0.730772,-0.425364
3,-0.958565,-0.542601,0.583916,-0.845588,-0.31183,1.240641,-0.587823,-0.442517,-0.477413,-0.019166,...,-0.189128,-0.367316,-1.695444,-0.924188,2.677904,-1.297410,0.897048,-0.634586,-0.730772,-0.425364
4,1.561116,-0.542601,0.954465,-0.845588,-0.31183,-0.806035,-0.587823,-0.442517,2.094620,-0.019166,...,-0.617485,-1.198881,-1.695444,-1.288303,-0.373426,0.770766,0.897048,-0.634586,-0.730772,2.350925


In [ ]:
# carregando a lista de variáveis que passaram pelo Feature Sepection (utilizadas no treinamento do modelo)
with open('/content/drive/MyDrive/1 - Aulas PoD Academy/Cientista de Dados POD/Aulas/Case Porto Seguro/prd_list_features.pkl', 'rb') as f:
    loaded_features = pickle.load(f)

loaded_features

['ps_ind_03',
 'ps_ind_15',
 'ps_reg_02',
 'ps_reg_03',
 'ps_car_11_cat',
 'ps_car_13',
 'ps_car_14',
 'ps_calc_01',
 'ps_calc_02',
 'ps_calc_03',
 'ps_calc_10',
 'ps_calc_11',
 'ps_calc_14']

In [ ]:
# criando os dataframes abt de treino e teste
abt_train = df_train_05[loaded_features]
abt_test = df_test_05[loaded_features]

In [ ]:
abt_train.head()

,ps_ind_03,ps_ind_15,ps_reg_02,ps_reg_03,ps_car_11_cat,ps_car_13,ps_car_14,ps_calc_01,ps_calc_02,ps_calc_03,ps_calc_10,ps_calc_11,ps_calc_14
0,0.213366,1.043812,-0.592188,-5.639059e-01,-1.522272,0.314372,-0.088932,0.522242,0.176951,-0.869620,-1.182005,1.524300,0.168156
1,0.954465,-1.212218,-0.097692,-4.103651e-01,-1.310157,-0.864661,0.317958,-0.522822,-1.216918,-0.521332,-0.493106,-1.045842,0.532271
2,1.695565,1.325816,-1.086683,-3.550803e-16,-0.067774,-0.763304,-0.623732,0.173887,0.873885,-1.217908,-0.493106,-0.617485,-0.195959
3,-0.898283,0.197801,-0.592188,-1.002463e+00,1.265515,-1.202386,-1.812583,0.522242,1.570820,-1.217908,-2.215353,-1.474199,0.532271
4,-1.639382,0.479805,0.396803,-1.715147e-01,0.598870,-1.100524,-0.218621,-0.174467,0.525418,-1.566196,1.229141,-1.045842,-1.652418


In [ ]:
# Carregando modelo campeão

with open('/content/drive/MyDrive/1 - Aulas PoD Academy/Cientista de Dados POD/Aulas/Case Porto Seguro/best_model_lightgbm.pkl', 'rb') as f:
  loaded_model = pickle.load(f)

loaded_model

LGBMClassifier(lambda_l1=1, lambda_l2=1.5, max_depth=3, min_data_in_leaf=300,
               num_leaves=3, reg_alpha=0.1)

In [ ]:
# Escorando base de treino
predprob = loaded_model.predict_proba(abt_train)
predict = loaded_model.predict(abt_train)
abt_train['score_1']=predprob[:,1]
abt_train['class']=predict

# Escorando base de teste
predprob = loaded_model.predict_proba(abt_test)
predict = loaded_model.predict(abt_test)
abt_test['score_1']=predprob[:,1]
abt_test['class']=predict

[LightGBM] [Warning] min_data_in_leaf is set=300, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=300
[LightGBM] [Warning] lambda_l2 is set=1.5, reg_lambda=0.0 will be ignored. Current value: lambda_l2=1.5
[LightGBM] [Warning] lambda_l1 is set=1, reg_alpha=0.1 will be ignored. Current value: lambda_l1=1
[LightGBM] [Warning] min_data_in_leaf is set=300, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=300
[LightGBM] [Warning] lambda_l2 is set=1.5, reg_lambda=0.0 will be ignored. Current value: lambda_l2=1.5
[LightGBM] [Warning] lambda_l1 is set=1, reg_alpha=0.1 will be ignored. Current value: lambda_l1=1
[LightGBM] [Warning] min_data_in_leaf is set=300, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=300
[LightGBM] [Warning] lambda_l2 is set=1.5, reg_lambda=0.0 will be ignored. Current value: lambda_l2=1.5
[LightGBM] [Warning] lambda_l1 is set=1, reg_alpha=0.1 will be ignored. Current value: lambda_l1=1
[LightGBM] [Warning

In [ ]:
abt_train = abt_train.merge(df_train_00[['id','target']], left_index=True, right_index=True, how='inner')
abt_test = abt_test.merge(df_test_00[['id']], left_index=True, right_index=True, how='inner')

In [ ]:
abt_test.head()

,ps_ind_03,ps_ind_15,ps_reg_02,ps_reg_03,ps_car_11_cat,ps_car_13,ps_car_14,ps_calc_01,ps_calc_02,ps_calc_03,ps_calc_10,ps_calc_11,ps_calc_14,score_1,class,id
0,1.325015,1.325816,-0.344940,-9.084966e-01,0.083736,-0.638793,-0.513267,-1.219531,1.222352,0.523533,0.195793,-1.902556,1.624615,0.423741,0,0
1,0.213366,-0.648211,0.149555,-3.934629e-01,1.235213,-0.920289,-0.372535,-0.174467,0.176951,-0.173043,-0.493106,-1.474199,0.896386,0.521214,1,1
2,-0.527733,0.761809,-1.086683,6.968516e-02,-1.007137,0.370281,0.540227,0.522242,0.525418,0.523533,1.229141,-0.617485,-1.288303,0.454611,0,2
3,0.583916,-0.930214,-0.592188,-3.550803e-16,-0.673815,-0.716454,0.152728,-1.219531,0.176951,0.175245,1.573590,-0.189128,-0.924188,0.408975,0,3
4,0.954465,-0.930214,-0.097692,-2.450343e-01,1.174609,-0.000636,0.235733,1.567305,0.525418,1.220110,1.229141,-0.617485,-1.288303,0.540581,1,4


#### Preparando o arquivo para enviar para a competição

In [ ]:
# verificando o formato que eles querem
df_submission = pd.read_csv('/content/drive/MyDrive/1 - Aulas PoD Academy/Cientista de Dados POD/Aulas/Case Porto Seguro/sample_submission.csv')
df_submission.head()

,id,target
0,0,0.0364
1,1,0.0364
2,2,0.0364
3,3,0.0364
4,4,0.0364


In [ ]:
# preparando para enviar para o Kaggle
abt_envio_kaggle = abt_test[['id','score_1']]
abt_envio_kaggle.rename(columns={'score_1':'target'},inplace=True)
abt_envio_kaggle.head()

,id,target
0,0,0.423741
1,1,0.521214
2,2,0.454611
3,3,0.408975
4,4,0.540581


In [ ]:
abt_envio_kaggle.shape

(892816, 2)

In [ ]:
# salvando o arquivo
abt_envio_kaggle.to_csv('/content/drive/MyDrive/1 - Aulas PoD Academy/Cientista de Dados POD/Aulas/Case Porto Seguro/abt_scored_01.csv', index=False)

Pronto!

Agora é só submeter o arquivo no Kaggle.